### Measuring transaction costs and trading volume

In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy.stats as scipy
sys.path.append("utils")
import dynamic_backtesting as db
from compare_strategies import calc_transaction_costs as ctc
plt.style.use('seaborn')
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=["black", "green",  "deepskyblue", "orange","grey", "lime", "purple", "Gold", "brown", "blue", "pink"]) 

In [ ]:
assets = {"EEM": "Emerging Markets",
           "IVV": "S&P 500",
           "IEV": "MSCI Europe",
           "IXN": "Global Tech",
           "IYR": "US Real Estate",
           "IXG": "Global Financials",
           "EXI": "Global Industrials",
           "GC=F": "Gold Futures", 
           "BZ=F": "Brent Crude Oil Futures",
           "HYG": "High-Yield Corporate Bonds",
           "TLT": "20+ Year Treasury Bonds"}
tickers = list(assets.keys())
asset_names = list(assets.values())
stocks = tickers[:7]
commodities = tickers[7:9]
bonds = tickers[9:]

In [ ]:
models = ['sGARCH10', 'sGARCH11', 'gjrGARCH11']
tuning_gamma_Ds = [1.52e-6, 1.3e-5, 1e-4]

In [ ]:
def remove_extreme(num, tolerance):
    if num > tolerance:
        return tolerance
    if num < 0:
        return np.nan
    return num

In [ ]:
def calc_TC(model, tuning_gamma_D, assets):
    v_t_s, out_of_sample, in_sample, Omega_ts = db.garch_with_trading_cost(tickers=assets, model_type=model, tuning_gamma_D=tuning_gamma_D)
    out_of_sample = out_of_sample.divide(100)
    TC = ctc(v_t_s, out_of_sample, Omega_ts)
    TC = [remove_extreme(x, 1) for x in TC]
    return TC

In [ ]:
def calc_multiple_TC(tickers):
    TCs = []
    for i, model in enumerate(models):
        row = []
        for j, tuning_gamma_D in enumerate(tuning_gamma_Ds):
            row.append(calc_TC(model, tuning_gamma_D, tickers))
        TCs.append(row)
    TCs_mean = [[np.nanmean(TC) for TC in TC_row] for TC_row in TCs]
    return TCs, TCs_mean

In [ ]:
TCs_all, TCs_all_mean = calc_multiple_TC(tickers)

In [ ]:
TCs_bonds, TCs_bonds_mean = calc_multiple_TC(bonds)

In [ ]:
TCs_stocks, TCs_stocks_mean = calc_multiple_TC(stocks)

In [ ]:
TCs_com, TCs_com_mean = calc_multiple_TC(commodities)

In [ ]:
def annualize_TC(TC):
    TC = np.array(TC)
    TC += 1
    TC_cum = np.cumprod(TC)
    last_TC = pd.Series(TC_cum).ffill().iloc[-1]
    num_non_nan_periods = len(TC_cum)-np.isnan(TC).sum()
    ann_return = (last_TC ** (250/num_non_nan_periods))-1
    return ann_return

In [ ]:
TC_all_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_all]

In [ ]:
TC_bonds_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_bonds]

In [ ]:
TC_stocks_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_stocks]

In [ ]:
TC_com_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_com]